# 40 kotlin-dataframe puzzles
inspired by [100 pandas puzzles](https://github.com/ajcr/100-pandas-puzzles)

Challenges taken from:
https://github.com/Kotlin/dataframe/blob/master/examples/jupyter-notebooks/puzzles/40%20puzzles.ipynb

## Importing kotlin-dataframe
### Getting started
Difficulty: easy

**1.** Import kotlin-dataframe

In [1]:
%useLatestDescriptors
%use dataframe

## DataFrame Basics
### A few of the fundamental routines for selecting, sorting, adding and aggregating data in DataFrames
Difficulty: easy

Consider the following columns:
```[kotlin]
val animal by columnOf("cat", "cat", "snake", "dog", "dog", "cat", "snake", "cat", "dog", "dog")
val age by columnOf(2.5, 3.0, 0.5, Double.NaN, 5.0, 2.0, 4.5, Double.NaN, 7, 3)
val visits by columnOf(1, 3, 2, 3, 2, 3, 1, 1, 2, 1)
val priority by columnOf("yes", "yes", "no", "yes", "no", "no", "no", "yes", "no", "no")
```
**2.** Create a DataFrame df from this columns.

In [28]:
val animal by columnOf("cat", "cat", "snake", "dog", "dog", "cat", "snake", "cat", "dog", "dog")
val age by columnOf(2.5, 3.0, 0.5, Double.NaN, 5.0, 2.0, 4.5, Double.NaN, 7, 3)
val visits by columnOf(1, 3, 2, 3, 2, 3, 1, 1, 2, 1)
val priority by columnOf("yes", "yes", "no", "yes", "no", "no", "no", "yes", "no", "no")

val df = dataFrameOf(animal, age, visits, priority).convert("age").toDouble()

**3.** Display a summary of the basic information about this DataFrame and its data.

In [32]:
df.describe()

DataFrame: rowsCount = 4, columnsCount = 12

**4.** Return the first 3 rows of the DataFrame df.

In [33]:
df.head(3)

DataFrame: rowsCount = 3, columnsCount = 4

**5.** Select "animal" and "age" columns from the DataFrame df.

In [34]:
df.select { animal and age }

DataFrame: rowsCount = 10, columnsCount = 2

**6.** Select the data in rows [3, 4, 8] and in columns ["animal", "age"].

In [36]:
df[3, 4, 8].select { animal and age }

DataFrame: rowsCount = 3, columnsCount = 2

**7.** Select only the rows where the number of visits is grater than 2.

In [37]:
df.filter { visits > 2 }

DataFrame: rowsCount = 3, columnsCount = 4

**8.** Select the rows where the age is missing, i.e. it is NaN.

In [38]:
df.filter { age.isNaN() }

DataFrame: rowsCount = 2, columnsCount = 4

**9.** Select the rows where the animal is a cat and the age is less than 3.

In [39]:
df.filter { animal == "cat" && age < 3 }

DataFrame: rowsCount = 2, columnsCount = 4

**10.** Select the rows where age is between 2 and 4 (inclusive).

In [41]:
df.filter { age in 2.0..4.0}

DataFrame: rowsCount = 4, columnsCount = 4

**11.** Change tha age in row 5 to 1.5

In [42]:
df.update { age }.at(5).with { 1.5 }

DataFrame: rowsCount = 10, columnsCount = 4

**12.** Calculate the sum of all visits in df (i.e. the total number of visits).

In [43]:
df.sum { visits }

19

**13.** Calculate the mean age for each different animal in df.

In [46]:
df.groupBy { animal }.mean("age", skipNA = true )

DataFrame: rowsCount = 3, columnsCount = 2

**14.** Append a new row to df with your choice of values for each column. Then delete that row to return the original DataFrame.

In [49]:
val mod = df.append("dog", 1.0, 5, "no")
mod.dropLast()

DataFrame: rowsCount = 10, columnsCount = 4

**15.** Count the number of each type of animal in df.

In [58]:
df.groupBy { animal }.count()

DataFrame: rowsCount = 3, columnsCount = 2

**16.** Sort df first by the values in the 'age' in decending order, then by the value in the 'visits' column in ascending order.

In [61]:
df.sortBy { age.desc() and visits }

DataFrame: rowsCount = 10, columnsCount = 4

**17.** The 'priority' column contains the values 'yes' and 'no'. Replace this column with a column of boolean values: 'yes' should be True and 'no' should be False.

In [63]:
df.convert { priority }.with { it == "yes" }

DataFrame: rowsCount = 10, columnsCount = 4

**18.** In the 'animal' column, change the 'dog' entries to 'corgi'.

In [64]:
df.update { animal }.where { it == "dog"}.with { "corgi" }

DataFrame: rowsCount = 10, columnsCount = 4

**19.** For each animal type and each number of visits, find the mean age. In other words, each row is an animal, each column is a number of visits and the values are the mean ages.

In [68]:
df.groupBy { animal }.pivot { visits }
    .mean(skipNA = true) { age }

DataFrame: rowsCount = 3, columnsCount = 2

## DataFrame: beyond the basics
### Slightly trickier: you may need to combine two or more methods to get the right answer
Difficulty: medium

The previous section was tour through some basic but essential DataFrame operations. Below are some ways that you might need to cut your data, but for which there is no single "out of the box" method.

**20.** You have a DataFrame df with a column 'A' of integers. For example:
```kotlin
val df = dataFrameOf("A")(1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7)
```
How do you filter out rows which contain the same integer as the row immediately above?

You should be left with a column containing the following values:
```
1, 2, 3, 4, 5, 6, 7
```

In [76]:
val df = dataFrameOf("A")(1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7)

In [78]:
df.filter { it.A != it.prev()?.A }

DataFrame: rowsCount = 7, columnsCount = 1

**21.** Given a DataFrame of random numetic values:
```kotlin
val df = dataFrameOf("a", "b", "c").randomDouble(5) // this is a 5x3 DataFrame of double values
```

how do you subtract the row mean from each element in the row?

In [80]:
val df = dataFrameOf("a", "b", "c").randomDouble(5) // this is a 5x3 DataFrame of double 
df

DataFrame: rowsCount = 5, columnsCount = 3

In [81]:
// aggregate all columns of type Double and the map their value
// with the `with` method
df.update { colsOf<Double>() }.with { it - rowMean() }

DataFrame: rowsCount = 5, columnsCount = 3

**22.** Suppose you have DataFrame with 10 columns of real numbers, for example:
```kotlin
val names = ('a'..'j').map { it.toString() }
val df = dataFrameOf(names).randomDouble(5)
```

Which column of number has the smallest sum? Return that column's label.

In [84]:
val names = ('a'..'j').map { it.toString() }
val df = dataFrameOf(names).randomDouble(5)
df

DataFrame: rowsCount = 5, columnsCount = 10

In [93]:
df.sum().transpose().minBy("value")["name"]

c

**23.** How do you count how many unique rows a DataFrame has (i.e. ignore all rows that are duplicates)?

In [94]:
df.distinct().count()

5

**24.** In the cell below, you have a DataFrame `df` that consists of 10 columns of floating-point numbers. Exactly 5 entries in each row are NaN values.

For each row of the DataFrame, find the *column* which contains the *third* NaN value.

You should return a column of column labels: `e, c, d, h, d`

In [95]:
val nan = Double.NaN
val names = ('a'..'j').map { it.toString() }
val data = listOf(
    0.04,  nan,  nan, 0.25,  nan, 0.43, 0.71, 0.51,  nan,  nan,
    nan,  nan,  nan, 0.04, 0.76,  nan,  nan, 0.67, 0.76, 0.16,
    nan,  nan, 0.5 ,  nan, 0.31, 0.4 ,  nan,  nan, 0.24, 0.01,
    0.49,  nan,  nan, 0.62, 0.73, 0.26, 0.85,  nan,  nan,  nan,
    nan,  nan, 0.41,  nan, 0.05,  nan, 0.61,  nan, 0.48, 0.68
)
val df = dataFrameOf(names)(*data.toTypedArray())
df

DataFrame: rowsCount = 5, columnsCount = 10

In [101]:
df.mapToColumn("result") {
    // list of Pair<colName, rowColValue> for each row
    namedValuesOf<Double>()
        .filter { it.value.isNaN() }.drop(2) // pop the end
        .firstOrNull()?.name
}

DataColumn: name = "result", type = String, size = 5

**25.** A DataFrame has a column of groups 'grps' and and column of integer values 'vals':
```kotlin
val grps by column("a", "a", "a", "b", "b", "c", "a", "a", "b", "c", "c", "c", "b", "b", "c")
val vals by column(12, 345, 3, 1, 45, 14, 4, 52, 54, 23, 235, 21, 57, 3, 87)

val df = dataFrameOf(grps, vals)
```

For each group, find the sum of the three greatest values. You should end up with the answer as follows:
```
grps
a    409
b    156
c    345
```

In [104]:
val grps by columnOf("a", "a", "a", "b", "b", "c", "a", "a", "b", "c", "c", "c", "b", "b", "c")
val vals by columnOf(12, 345, 3, 1, 45, 14, 4, 52, 54, 23, 235, 21, 57, 3, 87)

val df = dataFrameOf(grps, vals)
df

DataFrame: rowsCount = 15, columnsCount = 2

In [117]:
df.groupBy { grps }.aggregate {
    it.vals.sortDesc().take(3).sum() into "sum"
}.flatten()

DataFrame: rowsCount = 3, columnsCount = 2

**26.** The DataFrame `df` constructed below has two integer columns 'A' and 'B'. The values in 'A' are between 1 and 100 (inclusive).

For each group of 10 consecutive integers in 'A' (i.e. `(0, 10]`, `(10, 20]`, ...), calculate the sum of the corresponding values in column 'B'.

The answer as follows:

```
A
(0, 10]      635
(10, 20]     360
(20, 30]     315
(30, 40]     306
(40, 50]     750
(50, 60]     284
(60, 70]     424
(70, 80]     526
(80, 90]     835
(90, 100]    852
```

In [126]:
import kotlin.random.Random

val random = Random(42)
val list = List(200) { random.nextInt(1, 101)}
val df = dataFrameOf("A", "B")(*list.toTypedArray())
df

... showing only top 20 of 100 rows DataFrame: rowsCount = 100, columnsCount = 2

In [134]:
// creating ranges
df.groupBy { A.map { (it - 1) / 10 } }
    .sum { B }
    .sortBy { A }
    .convert { A }.with { "(${it * 10}, ${it * 10 + 10}]"}

DataFrame: rowsCount = 10, columnsCount = 2

## DataFrames: harder problems

### These might require a bit of thinking outside the box...

Difficulty: hard

**27.** Consider a DataFrame `df` where there is an integer column 'X':
```kotlin
val df = dataFrameOf("X")(7, 2, 0, 3, 4, 2, 5, 0, 3 , 4)
```
For each value, count the difference back to the previous zero (or the start of the column, whichever is closer). These values should therefore be

```
[1, 2, 0, 1, 2, 3, 4, 0, 1, 2]
```

Make this a new column 'Y'.

In [135]:
val df = dataFrameOf("X")(7, 2, 0, 3, 4, 2, 5, 0, 3 , 4)

In [142]:
df.mapToColumn("Y") {
    // newValue<*>() is used to access value that was already
    // calculated for preceding row.
    if (it.X == 0) 0 else (it.prev()?.newValue() ?: 0) + 1
}

DataColumn: name = "Y", type = Int, size = 10

**28.** Consider the DataFrame constructed below which contains rows and columns of numerical data.

Create a list of the column-row index locations of the 3 largest values in this DataFrame. In thi case, the answer should be:
```
[(0, d), (2, c), (3, f)]
```

In [143]:
val names = ('a'..'h').map { it.toString() } // val names = (0..7).map { it.toString() }
val random = Random(30)
val list = List(64) { random.nextInt(1, 101) }
val df = dataFrameOf(names)(*list.toTypedArray())
df

DataFrame: rowsCount = 8, columnsCount = 8

In [159]:
df.add("idx") { it.index() }
    .gather { it.dropLast() }.into("name", "vals")
    .sortByDesc("vals").take(3)["idx", "name"]

DataFrame: rowsCount = 3, columnsCount = 2

**29.** You are given the DataFrame below with a column of group IDs, 'grps', and a column of corresponding integer values, 'vals'.

```kotlin
val random = Random(31)
val lab =  listOf("A", "B")

val vals by columnOf(List(15) { random.nextInt(-30, 30) })
val grps by columnOf(List(15) { lab[random.nextInt(0, 2)] })

val df = dataFrameOf(vals, grps)
```

Create a new column 'patched_values' which contains the same values as the 'vals' any negative values in 'vals' with the group mean:

```
vals    grps    patched_vals
 -17      B         21.0
  -7      B         21.0
  28      B         28.0
  16      B         16.0
 -21      B         21.0
  19      B         19.0
  -2      B         21.0
 -19      B         21.0
  16      A         16.0
   9      A         9.0
 -14      A         16.0
 -19      A         16.0
 -22      A         16.0
  -1      A         16.0
  23      A         23.0
```

**30.** Implement a rolling mean over groups with window size 3, which ignores NaN value. For example consider the following DataFrame:
```kotlin
val group by columnOf("a", "a", "b", "b", "a", "b", "b", "b", "a", "b", "a", "b")
val value by columnOf(1.0, 2.0, 3.0, Double.NaN, 2.0, 3.0, Double.NaN, 1.0, 7.0, 3.0, Double.NaN, 8.0)

val df = dataFrameOf(group, value)
df

group   value
a       1.0
a       2.0
b       3.0
b       NaN
a       2.0
b       3.0
b       NaN
b       1.0
a       7.0
b       3.0
a       NaN
b       8.0
```
The goal is:
```
1.000000
1.500000
3.000000
3.000000
1.666667
3.000000
3.000000
2.000000
3.666667
2.000000
4.500000
4.000000
```
E.g. the first window of size three for group 'b' has values 3.0, NaN and 3.0 and occurs at row index 5. Instead of being NaN the value in the new column at this row index should be 3.0 (just the two non-NaN values are used to compute the mean (3+3)/2)

## Date
Difficulty: easy/medium

**31.** Create a column Of LocalDate that contains each day of 2015 and column of random numbers.

In [2]:
import kotlinx.datetime.*

**32.** Find the sum of the values in s for every Wednesday.

**33.** For each calendar month in s, find the mean of values.

**34.** For each group of four consecutive calendar months in s, find the date on which the highest value occurred.

**35.**  Create a column consisting of the third Thursday in each month for the years 2015 and 2016.

In [3]:
import java.time.temporal.WeekFields
import java.util.*

## Cleaning Data
### Making a DataFrame easier to work with
Difficulty: *easy/medium*

It happens all the time: someone gives you data containing malformed strings, lists and missing data. How do you tidy it up so you can get on with the analysis?

Take this monstrosity as the DataFrame to use in the following puzzles:
```kotlin
val fromTo = listOf("LoNDon_paris", "MAdrid_miLAN", "londON_StockhOlm", "Budapest_PaRis", "Brussels_londOn").toColumn("From_To")
val flightNumber = listOf(10045.0, Double.NaN, 10065.0, Double.NaN, 10085.0).toColumn("FlightNumber")
val recentDelays = listOf(listOf(23, 47), listOf(), listOf(24, 43, 87), listOf(13), listOf(67, 32)).toColumn("RecentDelays")
val airline = listOf("KLM(!)", "<Air France> (12)", "(British Airways. )", "12. Air France", "'Swiss Air'").toColumn("Airline")

val df = dataFrameOf(fromTo, flightNumber, recentDelays, airline)
```

It looks like this:
```
From_To             FlightNumber    RecentDelays    Airline
LoNDon_paris        10045.000000    [23, 47]        KLM(!)
MAdrid_miLAN        NaN             []              {Air France} (12)
londON_StockhOlm    10065.000000    [24, 43, 87]    (British Airways. )
Budapest_PaRis      NaN             [13]            12. Air France
Brussels_londOn     10085.000000    [67, 32]        'Swiss Air'
```

**36.** Some values in the FlightNumber column are missing (they are NaN). These numbers are meant to increase by 10 with each row, so 10055 and 10075 need to be put in place. Modify df to fill in these missing numbers and make the column an integer column (instead of a float column).

**37.** The **From_To** column would be better as two separate columns! Split each string on the underscore delimiter **_** to give a new two columns. Assign the correct names 'From' and 'To' to this columns.

**38.** Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame 'temp'. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

**39.** In the **Airline** column, you can see some extra punctuation and symbols have appeared around the airline names. Pull out just the airline name. E.g. `'(British Airways. )'` should become `'British Airways'`.

**40.** In the **RecentDelays** column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be `null`.

Expand the column of lists into columns named 'delays_' and replace the unwanted RecentDelays column in `df` with 'delays'.